### Drawback of traditional ML approach: Không giữ được order meaning của chuỗi axit amin

#### Sol: sử dụng mô hình esm2 650M tham số để embedding các chuỗi acid amin

## Load model

In [ ]:
%pip install transformers

In [3]:
import torch
import numpy as np
import pandas as pd
from Bio import SeqIO
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on device: {device}")

Running on device: cuda


In [7]:
model_name = "facebook/esm2_t33_650M_UR50D"

print(f"Loading model: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval() #read-only to save VRAM

Loading model: facebook/esm2_t33_650M_UR50D...


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 1280, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-32): 33 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=1280, out_features=1280, bias=True)
            (key): Linear(in_features=1280, out_features=1280, bias=True)
            (value): Linear(in_features=1280, out_features=1280, bias=True)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=1280, out_features=1280, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
          (dense): Linear(in_features=1280, out_features=5120, bias=True)
        )
        (output): EsmOut

## Feature extraction

In [8]:
def extract_embeddings(fasta_path, save_name, batch_size=8, limit=None):
    """
    Read fasta file -> Run through ESM2 -> output: .npy
    save_name: output file
    """
    ids = []
    sequences = []

    print(f"Reading file: {fasta_path}")
    for i, record in enumerate(SeqIO.parse(fasta_path, "fasta")):
        if limit and i >= limit: break

        #clean id
        pid = str(record.id)
        if "|" in pid:
            pid = pid.split("|")[1]

        ids.append(pid)
        #esm2 limit 1024 token
        sequences.append(str(record.seq)[:1022])

    print(f"{len(sequences)} Proteins")

    #batching
    embeddings = []
    print("Creating embeddings...")

    for i in tqdm(range(0, len(sequences), batch_size)):
        batch_seqs = sequences[i : i + batch_size]

        #tokenize
        inputs = tokenizer(batch_seqs, return_tensors="pt", padding=True, truncation=True, max_length=1024)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        #mean pooling
        last_hidden_state = outputs.last_hidden_state
        mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * mask, 1)
        sum_mask = torch.clamp(mask.sum(1), min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask

        embeddings.append(mean_embeddings.cpu().numpy())

    final_embeddings = np.vstack(embeddings)

    np.save(f"/workspace/data/Embeddings/{save_name}.npy", final_embeddings)
    np.save(f"/workspace/data/Embeddings/{save_name}_ids.npy", ids)

    return ids, final_embeddings

In [ ]:
train_fasta = "/workspace/data/Train/train_sequences.fasta"

train_ids, X_train = extract_embeddings(
    train_fasta, 
    save_name="train_650M", 
    batch_size=8
)

print(f"Shape X_train: {X_train.shape}")

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

#prepare labels
train_terms = pd.read_csv("/workspace/data/Train/train_terms.tsv", sep="\t", usecols=["EntryID", "term"])

train_ids_set = set(train_ids)
train_terms_filtered = train_terms[train_terms["EntryID"].isin(train_ids_set)]

top_n = 1500
top_terms = train_terms_filtered["term"].value_counts().head(top_n).index.tolist()

Y_matrix = train_terms_filtered[train_terms_filtered["term"].isin(top_terms)] \
            .pivot_table(index="EntryID", columns="term", aggfunc="size", fill_value=0)
Y_train = Y_matrix.reindex(train_ids).fillna(0).astype(int)

In [ ]:
X_tr, X_val, Y_tr, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

In [ ]:
clf = RidgeClassifier(alpha=1.0)
clf.fit(X_tr, Y_tr)

In [ ]:
Y_pred_val = clf.predict(X_val)
score = f1_score(Y_val, Y_pred_val, average='micro')
print(f"Local F1-Score: {score:.4f}")

In [ ]:
# --- Cell chẩn đoán lỗi ---
import numpy as np
from sklearn.metrics import precision_score, recall_score

# 1. Kiểm tra xem mô hình có dự đoán ra cái gì không?
print(f"Tổng số mẫu trong tập Val: {Y_val.shape[0]}")
print(f"Tổng số nhãn cần dự đoán: {Y_val.shape[0] * Y_val.shape[1]}")
print(f"Số lượng nhãn 1 (Thực tế): {Y_val.sum()}")
print(f"Số lượng nhãn 1 (Mô hình dự đoán): {Y_pred_val.sum()}")

# 2. Nếu số dự đoán quá thấp (gần bằng 0), ta cần hạ ngưỡng (Threshold)
print("\n--- Thử chỉnh ngưỡng thủ công ---")
# Lấy điểm số thô thay vì nhãn cứng 0/1
decision_scores = clf.decision_function(X_val) 

# Thử các ngưỡng khác nhau
for thr in [0, -0.5, -1.0]: # Ridge score có thể âm
    y_pred_new = (decision_scores > thr).astype(int)
    new_f1 = f1_score(Y_val, y_pred_new, average='micro')
    print(f"Ngưỡng {thr}: F1-Score = {new_f1:.4f}")

In [11]:
import os
import gc
import torch
import numpy as np
from Bio import SeqIO
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

FASTA_PATH = "/workspace/data/Test/testsuperset.fasta" 
SAVE_DIR = "/workspace/data/Embeddings/embeddings_chunks"
MODEL_NAME = "facebook/esm2_t33_650M_UR50D"

CHUNK_SIZE = 5000  
BATCH_SIZE = 8    

# --- SETUP ---
os.makedirs(SAVE_DIR, exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

def process_and_save(seqs, ids, part_idx):
    embeddings = []
    for i in range(0, len(seqs), BATCH_SIZE):
        batch_seqs = seqs[i : i + BATCH_SIZE]
        inputs = tokenizer(batch_seqs, return_tensors="pt", padding=True, truncation=True, max_length=1024)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_state = outputs.last_hidden_state
            mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * mask, 1)
            sum_mask = torch.clamp(mask.sum(1), min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            
        embeddings.append(mean_embeddings.cpu().numpy())
    
    final_emb = np.vstack(embeddings)
    np.save(f"{SAVE_DIR}/test_part_{part_idx}.npy", final_emb)
    np.save(f"{SAVE_DIR}/test_ids_{part_idx}.npy", ids)

sequences = []
ids = []
part_counter = 0

pbar = tqdm(total=224309) 

for record in SeqIO.parse(FASTA_PATH, "fasta"):
    save_path_check = f"{SAVE_DIR}/test_part_{part_counter}.npy"
    
    if os.path.exists(save_path_check):
        sequences.append(1) 
        if len(sequences) >= CHUNK_SIZE:
            sequences = [] 
            ids = []
            part_counter += 1
            pbar.update(CHUNK_SIZE)
        continue

    pid = str(record.id).split("|")[1] if "|" in str(record.id) else str(record.id)
    ids.append(pid)
    sequences.append(str(record.seq)[:1022])
    pbar.update(1)
    
    if len(sequences) >= CHUNK_SIZE: 
        process_and_save(sequences, ids, part_counter)
        part_counter += 1
        
        sequences = []
        ids = []
        gc.collect()

if len(sequences) > 0:
    process_and_save(sequences, ids, part_counter)

print("Finished")

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 224309/224309 [1:46:48<00:00, 47.25it/s]

Finished


In [ ]:
import numpy as np
import pandas as pd
import glob
import gc
from tqdm import tqdm
from sklearn.linear_model import RidgeClassifier

TRAIN_EMB_PATH = "/workspace/data/Embeddings/train_650M.npy" 
TRAIN_IDS_PATH = "/workspace/data/Embeddings/train_650M_ids.npy"
TRAIN_TERMS_PATH = "/workspace/data/Train/train_terms.tsv"
TEST_CHUNKS_DIR = "/workspace/data/Embeddings/embeddings_chunks"
OUTPUT_FILE = "submission_level4_FINAL_fixed.tsv"

# Re-training
print("Loading data and training model...")
X_train = np.load(TRAIN_EMB_PATH)
train_ids = np.load(TRAIN_IDS_PATH, allow_pickle=True)

train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep="\t", usecols=["EntryID", "term"])
top_n = 1500
top_terms = train_terms["term"].value_counts().head(top_n).index.tolist()
train_ids_set = set(train_ids)
train_terms_filtered = train_terms[train_terms["EntryID"].isin(train_ids_set) & train_terms["term"].isin(top_terms)]

Y_matrix = train_terms_filtered.pivot_table(index="EntryID", columns="term", aggfunc="size", fill_value=0)
Y_train = Y_matrix.reindex(train_ids).fillna(0).astype(int)
terms_columns = Y_train.columns 

clf = RidgeClassifier(alpha=1.0)
clf.fit(X_train, Y_train)

del X_train, Y_train, train_terms, train_terms_filtered, Y_matrix
gc.collect()

print("Outputting (Top K)...")

chunk_files = sorted(glob.glob(f"{TEST_CHUNKS_DIR}/test_part_*.npy"), 
                     key=lambda x: int(x.split('_')[-1].replace('.npy','')))

output_lines = []
TOP_K = 50   
THRESHOLD = 0.01 #increased threshold

for f_path in tqdm(chunk_files):
    X_chunk = np.load(f_path)
    id_path = f_path.replace("test_part_", "test_ids_")
    ids_chunk = np.load(id_path, allow_pickle=True)
    
    #predicting
    decision_scores = clf.decision_function(X_chunk)
    probs = 1 / (1 + np.exp(-decision_scores))
    
    for i, pid in enumerate(ids_chunk):
        prob_row = probs[i]
        
        # 1. thresholding
        mask = prob_row > THRESHOLD
        if not np.any(mask):
            indices = np.argsort(prob_row)[-5:]
        else:
            candidates = np.where(mask)[0]
            
            #2: only select the top 50
            if len(candidates) > TOP_K:
                # get candidate scores
                cand_probs = prob_row[candidates]
                # sort for top k
                top_k_local_idx = np.argsort(cand_probs)[-TOP_K:]
                indices = candidates[top_k_local_idx]
            else:
                indices = candidates
            
        for idx in indices:
            term = terms_columns[idx]
            score = prob_row[idx]
            output_lines.append(f"{pid}\t{term}\t{score:.3f}")
            
    del X_chunk, ids_chunk, decision_scores, probs
    gc.collect()

# Output file
print(f"💾 Saving {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("Finished")

#### Score: 0.192

## Improvement

### GO Hierarchy: Ridge classifier đang học các nhãn 1 cách độc lập, nhãn con có thể có score cao, nhưng những nhãn cha chung chung thì score lại thấp

### Sol: Ensemble: Mix với naive approach 

In [1]:
import pandas as pd
from tqdm import tqdm

LEVEL4_FILE = "/workspace/notebooks/submission_level4_FINAL_fixed.tsv"
NAIVE_FILE = "/workspace/notebooks/submission_naive.tsv"
OUTPUT_FILE = "/workspace/notebooks/submission_ensemble_boosted.tsv"

print("Reading ESM2 output file....")
preds_l4 = {}

try:
    with open(LEVEL4_FILE) as f:
        for line in tqdm(f):
            parts = line.strip().split('\t')
            if len(parts) < 3: continue
            # Key : (ProteinID, GO_Term)
            key = (parts[0], parts[1])
            preds_l4[key] = float(parts[2])
except FileNotFoundError:
    print(f"File not found: {LEVEL4_FILE}.")
    raise

print("Reading file Naive...")
preds_naive = {}
try:
    with open(NAIVE_FILE) as f:
        for line in tqdm(f):
            parts = line.strip().split('\t')
            if len(parts) < 3: continue
            key = (parts[0], parts[1])
            preds_naive[key] = float(parts[2])
except FileNotFoundError:
    print(f"File not found {NAIVE_FILE}")
    raise

print("Ensembling...")

#Select all id-term pairs in 2 files
all_keys = set(preds_l4.keys()) | set(preds_naive.keys())
output_lines = []

W_L4 = 0.6
W_NAIVE = 0.4

for key in tqdm(all_keys):
    pid, term = key
    
    # Lấy điểm số, nếu file nào không có thì coi là 0
    score_l4 = preds_l4.get(key, 0.0)
    score_naive = preds_naive.get(key, 0.0)
    
    # Công thức cộng gộp
    final_score = (score_l4 * W_L4) + (score_naive * W_NAIVE)
    
    # Chỉ ghi những dòng có điểm số > 0.001 để file đỡ nặng
    if final_score > 0.001:
        output_lines.append(f"{pid}\t{term}\t{final_score:.3f}")

# Ghi ra file
print(f"Saving: {OUTPUT_FILE}")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print(f"Finisehd.")

Reading ESM2 output file....


11215450it [00:07, 1548396.54it/s]


Reading file Naive...


10093905it [00:05, 1690414.90it/s]


Ensembling...


100%|██████████| 17589919/17589919 [00:20<00:00, 844235.35it/s]


Saving: /workspace/notebooks/submission_ensemble_boosted.tsv
Finisehd.


### Ensemble ESM2 + BLAST/Diamond

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

LEVEL4_FILE = "/workspace/notebooks/submission_level4_FINAL_fixed.tsv"

DIAMOND_BIN = "/usr/bin/diamond"
TRAIN_FASTA = "/workspace/data/Train/train_sequences.fasta"
TEST_FASTA = "/workspace/data/Test/testsuperset.fasta"
DB_PATH = "/workspace/data/Traintrain_data.dmnd"
BLAST_RESULT = "/workspace/notebooks/diamond_results.tsv"
OUTPUT_FILE = "/workspace/notebooks/submission_hybrid_blast_esm2.tsv"

print("Runnign BLAST...")

#create db
if not os.path.exists(DB_PATH):
    print("Đang tạo database...")
    os.system(f"{DIAMOND_BIN} makedb --in {TRAIN_FASTA} -d {DB_PATH} --quiet")

#alignment
cmd = f"{DIAMOND_BIN} blastp -d {DB_PATH} -q {TEST_FASTA} -o {BLAST_RESULT} --sensitive --top 1 -f 6 qseqid sseqid pident"
os.system(cmd)
print("Finished blasting")

print("Handling output of BLAST...")
blast_preds = {}

# Load train terms để map nhãn
train_terms = pd.read_csv("/workspace/data/Train/train_terms.tsv", sep="\t", usecols=["EntryID", "term"])
train_terms_grouped = train_terms.groupby("EntryID")["term"].apply(list).to_dict()

# Đọc kết quả BLAST
df_blast = pd.read_csv(BLAST_RESULT, sep="\t", names=["test_id", "train_id", "pident"])

for _, row in tqdm(df_blast.iterrows(), total=len(df_blast)):
    # Clean ID (Sửa lỗi ID bị dính sp|...)
    test_id = str(row['test_id']).split("|")[1] if "|" in str(row['test_id']) else str(row['test_id'])
    train_id = str(row['train_id']).split("|")[1] if "|" in str(row['train_id']) else str(row['train_id'])
    
    score = row['pident'] / 100.0
    
    # Chỉ lấy những thằng giống nhau > 30% (Ngưỡng an toàn)
    if score < 0.3: continue
    
    if train_id in train_terms_grouped:
        for term in train_terms_grouped[train_id]:
            key = (test_id, term)
            # BLAST rất uy tín, nên giữ nguyên score cao
            blast_preds[key] = score

print("Reading file Level 4...")
esm_preds = {}
try:
    with open(LEVEL4_FILE) as f:
        for line in tqdm(f):
            parts = line.strip().split('\t')
            if len(parts) < 3: continue
            key = (parts[0], parts[1])
            esm_preds[key] = float(parts[2])
except FileNotFoundError:
    print(f"File not found {LEVEL4_FILE}.")
    raise

print("Mixing...")

all_keys = set(blast_preds.keys()) | set(esm_preds.keys())
output_lines = []

for key in tqdm(all_keys):
    pid, term = key
    
    s_blast = blast_preds.get(key, 0.0)
    s_esm = esm_preds.get(key, 0.0)
    
    # CHIẾN THUẬT QUAN TRỌNG:
    # Nếu BLAST tìm thấy -> Tin BLAST (vì nó so khớp chính xác)
    # Nếu BLAST không thấy -> Tin ESM (vì nó suy luận tốt)
    # => Lấy MAX
    final_score = max(s_blast, s_esm)
    
    output_lines.append(f"{pid}\t{term}\t{final_score:.3f}")

#GHI FILE
print(f"Saving {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print(f"Finished")

Runnign BLAST...


diamond v2.1.8.162 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: /workspace/notebooks
Percentage range of top alignment score to report hits: 1
Opening the database...  [0.06s]
Database: /workspace/data/Traintrain_data.dmnd (type: Diamond database, sequences: 82404, letters: 43327058)
Block size = 2000000000
Opening the input file...  [0.035s]
Opening the output file...  [0s]
Loading query sequences...  [0.215s]
Masking queries...  [0.141s]
Algorithm: Double-indexed
Building query histograms...  [0.513s]
Seeking in database...  [0s]
Loading reference sequences...  [0.051s]
Masking reference...  [0.061s]
Initializing temporary storage...  [0s]
Building reference histogra

Finished blasting
Handling output of BLAST...


100%|██████████| 262530/262530 [00:03<00:00, 72338.79it/s]


Reading file Level 4...


11215450it [00:07, 1560616.37it/s]


Mixing...


100%|██████████| 11968960/11968960 [00:12<00:00, 950665.04it/s]


Saving /workspace/notebooks/submission_hybrid_blast_esm2.tsv...
Finished


### Propagation

In [3]:
%pip install obonet networkx

Note: you may need to restart the kernel to use updated packages.


In [4]:
import networkx
import obonet
import pandas as pd
import numpy as np
from tqdm import tqdm

INPUT_FILE = "/workspace/notebooks/submission_level4_FINAL_fixed.tsv"
OUTPUT_FILE = "/workspace/notebooks/submission_level4_propagated.tsv"
OBO_PATH = "/workspace/data/Train/go-basic.obo"

In [5]:
print("Đọc cây phả hệ...")
graph = obonet.read_obo(OBO_PATH)

Đọc cây phả hệ...


In [6]:
print("Xây map quan hệ cha-con...")
ancestors_map = {}
for node in tqdm(graph.nodes()):
    # networkx trả về danh sách tổ tiên
    try:
        ancestors = networkx.descendants(graph, node) # Trong obonet, chiều mũi tên ngược (Con -> Cha)
        ancestors_map[node] = ancestors
    except:
        pass

Xây map quan hệ cha-con...


100%|██████████| 40122/40122 [00:00<00:00, 132343.17it/s]


In [7]:
print(f"Đang đọc {INPUT_FILE}...")
# Đọc vào Dict: {ProteinID: {Term: Score}}
preds = {}
with open(INPUT_FILE) as f:
    for line in tqdm(f):
        parts = line.strip().split('\t')
        if len(parts) < 3: continue
        pid, term, score = parts[0], parts[1], float(parts[2])
        
        if pid not in preds: preds[pid] = {}
        preds[pid][term] = score

Đang đọc /workspace/notebooks/submission_level4_FINAL_fixed.tsv...


11215450it [00:04, 2284548.44it/s]


In [8]:
print("Lan truyền ngược...")
final_lines = []

for pid, term_scores in tqdm(preds.items()):
    # term_scores là dict {Term: Score gốc}
    # new_scores sẽ chứa cả điểm của cha ông
    new_scores = term_scores.copy()
    
    for term, score in term_scores.items():
        # Lấy danh sách cha ông của term này
        if term in ancestors_map:
            parents = ancestors_map[term]
            for parent in parents:
                # Quy tắc: Điểm của cha = MAX(Điểm cha cũ, Điểm của con)
                current_parent_score = new_scores.get(parent, 0.0)
                new_scores[parent] = max(current_parent_score, score)
    
    # Ghi ra kết quả (Lại phải lọc Top K vì giờ nó phình to ra)
    # Sắp xếp giảm dần theo điểm
    sorted_terms = sorted(new_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Lấy Top 70 (tăng lên chút vì giờ có cả cha ông)
    for term, score in sorted_terms[:70]:
        final_lines.append(f"{pid}\t{term}\t{score:.3f}")

Lan truyền ngược...


100%|██████████| 224309/224309 [00:22<00:00, 9829.63it/s] 


In [9]:
print(f"Saving {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(final_lines))

print("Finished")

Saving /workspace/notebooks/submission_level4_propagated.tsv...
Finished


### Sử dụng KNN clf trên embeddings

In [21]:
import torch
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm
import glob

TRAIN_EMB_PATH = "/workspace/data/Embeddings/train_650M.npy"
TRAIN_IDS_PATH = "/workspace/data/Embeddings/train_650M_ids.npy"
TRAIN_TERMS_PATH = "/workspace/data/Train/train_terms.tsv"
TEST_CHUNKS_DIR = "/workspace/data/Embeddings/embeddings_chunks"
OUTPUT_FILE = "submission_knn_esm2.tsv"
TOP_K = 5  # Lấy 5 người hàng xóm giống nhất
device = "cuda" if torch.cuda.is_available() else "cpu"

In [22]:
print("Load Train Embeddings...")
# Load vector
X_train = np.load(TRAIN_EMB_PATH)
X_train = torch.from_numpy(X_train).to(device)

# Chuẩn hóa vector về đơn vị (để tính Cosine Similarity nhanh bằng phép nhân ma trận)
# Công thức: v = v / |v|
norm = X_train.norm(p=2, dim=1, keepdim=True)
X_train = X_train.div(norm)

# Load ID và Map Nhãn
print("Loading labels...")
train_ids = np.load(TRAIN_IDS_PATH, allow_pickle=True)
train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep="\t", usecols=["EntryID", "term"])

# Gom nhóm: TrainID -> Set(Terms)
train_labels_map = train_terms.groupby("EntryID")["term"].apply(set).to_dict()

# Map Index -> ID (để truy xuất nhanh từ kết quả KNN)
idx_to_trainid = {i: pid for i, pid in enumerate(train_ids)}

print("Train xong tren GPU!")

Load Train Embeddings...
Loading labels...
Train xong tren GPU!


In [23]:
print("Bắt đầu chạy KNN (Tìm hàng xóm)...")

chunk_files = sorted(glob.glob(f"{TEST_CHUNKS_DIR}/test_part_*.npy"), 
                     key=lambda x: int(x.split('_')[-1].replace('.npy','')))

output_lines = []

for f_path in tqdm(chunk_files):
    # 1. Load 1 cục Test lên GPU
    X_test_np = np.load(f_path)
    X_test = torch.from_numpy(X_test_np).to(device)
    
    # Chuẩn hóa Test
    norm_test = X_test.norm(p=2, dim=1, keepdim=True)
    X_test = X_test.div(norm_test)
    
    # Load ID Test tương ứng
    id_path = f_path.replace("test_part_", "test_ids_")
    ids_test = np.load(id_path, allow_pickle=True)
    
    # 2. Tính độ tương đồng (Matrix Multiplication)
    # [Batch, 1280] x [1280, All_Train] = [Batch, All_Train]
    # Đây là bước nặng nhất, nhưng GPU xử lý tốt
    sim_matrix = torch.mm(X_test, X_train.t())
    
    # 3. Lấy Top K hàng xóm
    # values: độ tương đồng (score), indices: vị trí của hàng xóm
    topk_values, topk_indices = torch.topk(sim_matrix, k=TOP_K, dim=1)
    
    # Chuyển về CPU để xử lý logic gán nhãn (Python xử lý dict nhanh hơn)
    topk_indices = topk_indices.cpu().numpy()
    topk_values = topk_values.cpu().numpy()
    
    # 4. Tổng hợp nhãn từ hàng xóm (Weighted Voting)
    for i, test_pid in enumerate(ids_test):
        # Dict lưu điểm số cho từng nhãn: {Term: Score}
        term_scores = {}
        
        for k in range(TOP_K):
            neighbor_idx = topk_indices[i, k]
            score = topk_values[i, k] # Độ giống nhau (ví dụ 0.95)
            
            neighbor_id = idx_to_trainid[neighbor_idx]
            
            # Nếu hàng xóm này có nhãn (có trong file train_terms)
            if neighbor_id in train_labels_map:
                neighbor_terms = train_labels_map[neighbor_id]
                for term in neighbor_terms:
                    # Cộng dồn điểm (Weighted Sum)
                    if term not in term_scores:
                        term_scores[term] = 0.0
                    term_scores[term] += score
        
        # Chuẩn hóa điểm số (Chia cho tổng trọng số hoặc K)
        # Ở đây ta chia cho K để score nằm trong khoảng 0-1
        # Hoặc đơn giản là giữ nguyên vì CAFA chấm rank
        
        # Lấy Top 50 nhãn điểm cao nhất để ghi file
        sorted_terms = sorted(term_scores.items(), key=lambda x: x[1], reverse=True)[:50]
        
        for term, total_score in sorted_terms:
            # Normalize heuristic: Score trung bình
            final_score = total_score / TOP_K 
            # Chỉ ghi nếu score đủ lớn
            if final_score > 0.01: 
                output_lines.append(f"{test_pid}\t{term}\t{final_score:.3f}")

    # Dọn dẹp bộ nhớ GPU
    del X_test, sim_matrix, topk_values, topk_indices
    torch.cuda.empty_cache()

print("Finished")

Bắt đầu chạy KNN (Tìm hàng xóm)...


100%|██████████| 45/45 [00:05<00:00,  8.25it/s]

Finished


In [24]:
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

Đang lưu submission_knn_esm2.tsv...


#### + Propagation

In [25]:
import networkx
import obonet
import pandas as pd
import numpy as np
from tqdm import tqdm

INPUT_FILE = "/workspace/notebooks/submission_knn_esm2.tsv" 
OUTPUT_FILE = "/workspace/notebooks/submission_knn_propagated.tsv"
OBO_PATH = "/workspace/data/Train/go-basic.obo"

In [26]:
# 1. Load cây Gene Ontology
print("Đọc file go-basic.obo...")
graph = obonet.read_obo(OBO_PATH)

# 2. Xây dựng bản đồ Cha-Con
print("Đang xây dựng quan hệ tổ tiên...")
ancestors_map = {}
# Chỉ quan tâm đến các node có quan hệ 'is_a' và 'part_of'
for node in tqdm(graph.nodes()):
    try:
        # Lấy tất cả tổ tiên của node hiện tại
        ancestors = networkx.descendants(graph, node)
        ancestors_map[node] = ancestors
    except:
        pass

# 3. Đọc file KNN
print(f"Đang đọc {INPUT_FILE}...")
preds = {} # {ProteinID: {Term: Score}}

with open(INPUT_FILE) as f:
    for line in tqdm(f):
        parts = line.strip().split('\t')
        if len(parts) < 3: continue
        pid, term, score = parts[0], parts[1], float(parts[2])
        
        if pid not in preds: preds[pid] = {}
        preds[pid][term] = score

# 4. Lan truyền điểm số (Propagation)
print("Đang lan truyền điểm số (Fill Parents)...")
output_lines = []

for pid, term_scores in tqdm(preds.items()):
    # Copy dict điểm cũ
    new_scores = term_scores.copy()
    
    # Duyệt qua từng term đang có
    for term, score in term_scores.items():
        # Nếu term này có tổ tiên
        if term in ancestors_map:
            parents = ancestors_map[term]
            for parent in parents:
                # Quy tắc cốt lõi: Điểm của Cha luôn >= Điểm của Con
                current_p_score = new_scores.get(parent, 0.0)
                new_scores[parent] = max(current_p_score, score)
    
    # Lọc và Ghi file
    # Sau khi lan truyền, số lượng nhãn sẽ phình to ra (vì thêm cha ông)
    # Ta chỉ lấy Top 70 nhãn điểm cao nhất để file không quá nặng
    sorted_terms = sorted(new_scores.items(), key=lambda x: x[1], reverse=True)[:70]
    
    for term, score in sorted_terms:
        # Lọc bớt rác: Chỉ lấy > 0.01
        if score > 0.01:
            output_lines.append(f"{pid}\t{term}\t{score:.3f}")

# 5. Lưu file
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("XONG!")

Đọc file go-basic.obo...
Đang xây dựng quan hệ tổ tiên...


100%|██████████| 40122/40122 [00:00<00:00, 158229.79it/s]


Đang đọc /workspace/notebooks/submission_knn_esm2.tsv...


4549325it [00:01, 2344697.58it/s]


Đang lan truyền điểm số (Fill Parents)...


100%|██████████| 224309/224309 [00:12<00:00, 18368.44it/s]


Đang lưu /workspace/notebooks/submission_knn_propagated.tsv...
XONG!


#### Score: 0.211

### Taking Taxonomy into consideration
Một protein của Người (Human) và của Vi khuẩn (E. coli) có thể nhìn hơi giống nhau (trình tự tương đồng), nhưng chức năng thì khác một trời một vực.

Nếu KNN tìm thấy "hàng xóm" là vi khuẩn để gán nhãn cho người, ta sẽ sai.

### Taxonomy-aware KNN

#### Strategy: ưu tiên cùng loài: Trong các neighbours của điểm cần label, 
Nếu Test Protein là Người (9606) và hàng xóm tìm được cũng là Người (9606) $\rightarrow$ Nhân điểm số lên 1.5 lần (Ưu tiên cực mạnh).Nếu khác loài $\rightarrow$ Giữ nguyên hoặc giảm nhẹ.

In [1]:
import torch
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm
import glob
import os

TRAIN_EMB_PATH = "/workspace/data/Embeddings/train_650M.npy"
TRAIN_IDS_PATH = "/workspace/data/Embeddings/train_650M_ids.npy"

TRAIN_TERMS_PATH = "/workspace/data/Train/train_terms.tsv"    
TRAIN_TAX_PATH   = "/workspace/data/Train/train_taxonomy.tsv"  
TEST_TAX_PATH    = "/workspace/data/Test/testsuperset-taxon-list.tsv" 

TEST_CHUNKS_DIR  = "/workspace/data/Embeddings/embeddings_chunks"
OUTPUT_FILE      = "/workspace/notebooks/submission_knn_tax_v2.tsv"

TOP_K = 10
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
print("Đang load Taxonomy Map...")

# 1. Load Train Taxonomy
# File này format: EntryID \t TaxID
try:
    df_train_tax = pd.read_csv(TRAIN_TAX_PATH, sep="\t")
    # Cần đảm bảo tên cột đúng. Thường là 'EntryID' và 'TaxonomyID' (hoặc 'taxon_id')
    # Ta convert sang dict cho nhanh: {PID: TaxID}
    cols = df_train_tax.columns
    train_tax_map = dict(zip(df_train_tax[cols[0]], df_train_tax[cols[1]]))
    print(f"Đã load {len(train_tax_map)} train TaxIDs.")
except FileNotFoundError:
    # Fallback nếu đường dẫn sai
    print(f"Không tìm thấy {TRAIN_TAX_PATH}, thử tìm ở thư mục gốc...")
    TRAIN_TAX_PATH = "/workspace/data/Train/train_taxonomy.tsv"
    df_train_tax = pd.read_csv(TRAIN_TAX_PATH, sep="\t")
    cols = df_train_tax.columns
    train_tax_map = dict(zip(df_train_tax[cols[0]], df_train_tax[cols[1]]))

# 2. Load Test Taxonomy
# File này thường không có header hoặc header lạ, ta cứ đọc thử
print(f"Đang đọc {TEST_TAX_PATH}...")
try:
    # Thử đọc header=None trước xem sao
    df_test_tax = pd.read_csv(TEST_TAX_PATH, sep="\t", header=None, names=["EntryID", "TaxID"])
    # Nếu dòng đầu tiên là chữ (header), ta drop nó đi
    if not str(df_test_tax.iloc[0, 1]).isdigit():
        df_test_tax = pd.read_csv(TEST_TAX_PATH, sep="\t")
        # Đổi tên cột cho chuẩn
        df_test_tax.columns = ["EntryID", "TaxID"]
    
    test_tax_map = dict(zip(df_test_tax["EntryID"], df_test_tax["TaxID"]))
    print(f"Đã load {len(test_tax_map)} test TaxIDs.")
except Exception as e:
    print(f"Lỗi đọc file Test Tax: {e}")
    raise

Đang load Taxonomy Map...
Đã load 82403 train TaxIDs.
Đang đọc /workspace/data/Test/testsuperset-taxon-list.tsv...
Đã load 8453 test TaxIDs.


In [3]:
print("Đang load Embeddings & Labels...")
# Load Train Embeddings
X_train = np.load(TRAIN_EMB_PATH)
X_train = torch.from_numpy(X_train).to(device)
norm = X_train.norm(p=2, dim=1, keepdim=True)
X_train = X_train.div(norm)

# Load Train IDs & Terms
train_ids = np.load(TRAIN_IDS_PATH, allow_pickle=True)
train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep="\t", usecols=["EntryID", "term"])
train_labels_map = train_terms.groupby("EntryID")["term"].apply(set).to_dict()
idx_to_trainid = {i: pid for i, pid in enumerate(train_ids)}

Đang load Embeddings & Labels...


In [4]:
print("Bắt đầu chạy Taxonomy-Aware KNN...")
chunk_files = sorted(glob.glob(f"{TEST_CHUNKS_DIR}/test_part_*.npy"), 
                     key=lambda x: int(x.split('_')[-1].replace('.npy','')))

output_lines = []

for f_path in tqdm(chunk_files):
    # Load 1 cục Test
    X_test_np = np.load(f_path)
    X_test = torch.from_numpy(X_test_np).to(device)
    norm_test = X_test.norm(p=2, dim=1, keepdim=True)
    X_test = X_test.div(norm_test)
    
    # Load ID
    id_path = f_path.replace("test_part_", "test_ids_")
    ids_test = np.load(id_path, allow_pickle=True)
    
    # Tính Cosine Sim
    sim_matrix = torch.mm(X_test, X_train.t())
    topk_values, topk_indices = torch.topk(sim_matrix, k=TOP_K, dim=1)
    
    topk_indices = topk_indices.cpu().numpy()
    topk_values = topk_values.cpu().numpy()
    
    # Xử lý Logic (Phần quan trọng nhất)
    for i, test_pid in enumerate(ids_test):
        # Lấy TaxID của con test này (Mặc định -1 nếu không tìm thấy)
        test_taxon = test_tax_map.get(test_pid, -1)
        
        term_scores = {}
        
        for k in range(TOP_K):
            neighbor_idx = topk_indices[i, k]
            raw_score = topk_values[i, k]
            
            neighbor_pid = idx_to_trainid[neighbor_idx]
            neighbor_taxon = train_tax_map.get(neighbor_pid, -2)
            
            # --- LOGIC THƯỞNG/PHẠT ---
            weight = 1.0
            
            # Nếu cả 2 đều có thông tin loài
            if test_taxon != -1 and neighbor_taxon != -2:
                if test_taxon == neighbor_taxon:
                    # CÙNG LOÀI: Tăng độ tin cậy lên 1.3 lần
                    weight = 1.3 
                else:
                    # KHÁC LOÀI: Giảm nhẹ (vì vẫn có thể bảo tồn chức năng)
                    weight = 0.9
            
            final_score = raw_score * weight
            
            # Cộng dồn điểm cho các chức năng
            if neighbor_pid in train_labels_map:
                for term in train_labels_map[neighbor_pid]:
                    if term not in term_scores: term_scores[term] = 0.0
                    term_scores[term] += final_score
        
        # Lấy kết quả (Top 60)
        sorted_terms = sorted(term_scores.items(), key=lambda x: x[1], reverse=True)[:60]
        
        for term, total_score in sorted_terms:
            # Normalize điểm số (chia cho số hàng xóm hoặc trọng số)
            # Ở đây chia cho TOP_K để đưa về range 0-1 (tương đối)
            norm_score = total_score / TOP_K
            
            # Nếu có boost thì norm_score có thể > 1.0 -> Clip về 1.0
            norm_score = min(norm_score, 1.0)
            
            if norm_score > 0.01:
                output_lines.append(f"{test_pid}\t{term}\t{norm_score:.3f}")

    del X_test, sim_matrix
    torch.cuda.empty_cache()

Bắt đầu chạy Taxonomy-Aware KNN...


100%|██████████| 45/45 [00:11<00:00,  3.99it/s]


In [5]:
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("XONG!")

Đang lưu /workspace/notebooks/submission_knn_tax_v2.tsv...
XONG!


#### + Propagation

In [6]:
%pip install -q obonet networkx

import networkx
import obonet
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

INPUT_FILE = "/workspace/notebooks/submission_knn_tax_v2.tsv"
OUTPUT_FILE = "/workspace/notebooks/submission_knn_tax_propagated.tsv"

OBO_PATH = "/workspace/data/Train/go-basic.obo"

# --- BƯỚC 1: XÂY DỰNG BẢN ĐỒ TỔ TIÊN ---
print("Đang đọc cây Gene Ontology...")
graph = obonet.read_obo(OBO_PATH)

print("Đang map quan hệ Con -> Cha Ông...")
ancestors_map = {}
# Chỉ quan tâm các node có trong graph
for node in tqdm(graph.nodes()):
    try:
        # networkx.descendants trong obonet trả về các node "cha/ông" (superclasses)
        ancestors = networkx.descendants(graph, node)
        ancestors_map[node] = ancestors
    except:
        pass

# --- BƯỚC 2: ĐỌC FILE DỰ ĐOÁN ---
print(f"Đang đọc {INPUT_FILE}...")
preds = {} # {PID: {Term: Score}}

try:
    with open(INPUT_FILE) as f:
        for line in tqdm(f):
            parts = line.strip().split('\t')
            if len(parts) < 3: continue
            pid, term, score = parts[0], parts[1], float(parts[2])
            
            if pid not in preds: preds[pid] = {}
            preds[pid][term] = score
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {INPUT_FILE}. Bạn đã chạy xong bước KNN Taxonomy chưa?")
    raise

# --- BƯỚC 3: LAN TRUYỀN (PROPAGATION) ---
print("Đang lan truyền điểm số (Fill Parents)...")
output_lines = []

for pid, term_scores in tqdm(preds.items()):
    # Copy dict điểm cũ sang dict mới để cập nhật
    new_scores = term_scores.copy()
    
    # Duyệt qua từng term gốc
    for term, score in term_scores.items():
        # Nếu term này có tổ tiên trong cây
        if term in ancestors_map:
            parents = ancestors_map[term]
            for parent in parents:
                # QUY TẮC VÀNG: Điểm Cha = Max(Điểm Cha cũ, Điểm Con)
                current_p_score = new_scores.get(parent, 0.0)
                if score > current_p_score:
                    new_scores[parent] = score
    
    # --- BƯỚC 4: LỌC & GHI ---
    # Sau khi lan truyền, số lượng nhãn sẽ tăng lên rất nhiều.
    # Ta cần lọc lấy Top K điểm cao nhất để file không bị quá nặng (gây lỗi submit).
    
    # Sắp xếp giảm dần theo điểm
    sorted_terms = sorted(new_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Lấy Top 75 (Đủ để bao phủ cả cha lẫn con)
    # Lấy nhiều hơn 75 file sẽ > 150MB, dễ bị lỗi timeout khi nộp
    for term, score in sorted_terms[:75]:
        # Chỉ lấy điểm > 0.01 (bỏ rác)
        if score > 0.01:
            output_lines.append(f"{pid}\t{term}\t{score:.3f}")

# Lưu file
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("-" * 30)
print(f"XONG!")

Note: you may need to restart the kernel to use updated packages.
Đang đọc cây Gene Ontology...
Đang map quan hệ Con -> Cha Ông...


100%|██████████| 40122/40122 [00:00<00:00, 153223.66it/s]


Đang đọc /workspace/notebooks/submission_knn_tax_v2.tsv...


7872336it [00:03, 2271903.00it/s]


Đang lan truyền điểm số (Fill Parents)...


100%|██████████| 224309/224309 [00:13<00:00, 16100.11it/s]


Đang lưu /workspace/notebooks/submission_knn_tax_propagated.tsv...
------------------------------
XONG!


#### Score: 0.234

#### Drawbacks of KNN: "Hai protein gần nhau thì chức năng gần nhau", thực tế có nhứng sequences chỉ có 1-2 acid amin giống nhau nhưng lại có chức năng tương tự and vice versa. Plus, KNN coi 1280 chiều là như nhau, major vote cũng chưa có trọng số.

### MLP

#### Xây dựng 1 mạng neuron đặt lên các vector embeddings. Đầu ra: 1500 nhãn (sigmoid)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import gc

TRAIN_EMB_PATH = "/workspace/data/Embeddings/train_650M.npy"
TRAIN_IDS_PATH = "/workspace/data/Embeddings/train_650M_ids.npy"
TRAIN_TERMS_PATH = "/workspace/data/Train/train_terms.tsv"
MODEL_SAVE_PATH = "/workspace/models/mlp_model.pth"
BATCH_SIZE = 128 # Tăng batch size lên vì embedding nhẹ
EPOCHS = 15      # Train kỹ một chút
LEARNING_RATE = 1e-3
TOP_LABELS = 1500 
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
print("Đang load dữ liệu lên RAM...")

# 1. Load Embeddings (X)
X_all = np.load(TRAIN_EMB_PATH)
ids_all = np.load(TRAIN_IDS_PATH, allow_pickle=True)

# 2. Load Labels (Y)
print("Đang xử lý nhãn...")
train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep="\t", usecols=["EntryID", "term"])

# Lọc IDs
ids_set = set(ids_all)
train_terms = train_terms[train_terms["EntryID"].isin(ids_set)]

# Lấy Top 1500 terms
top_term_list = train_terms["term"].value_counts().head(TOP_LABELS).index.tolist()
term_to_idx = {t: i for i, t in enumerate(top_term_list)}

# Tạo Y matrix (Numpy)
# Cách tạo nhanh hơn pivot_table:
Y_all = np.zeros((len(ids_all), TOP_LABELS), dtype=np.float32)
id_to_idx = {pid: i for i, pid in enumerate(ids_all)}

# Duyệt qua file terms để điền số 1 vào ma trận
# (Cách này tiết kiệm RAM hơn pivot table)
grouped = train_terms[train_terms["term"].isin(top_term_list)].groupby("EntryID")["term"].apply(list)

for pid, terms in tqdm(grouped.items()):
    if pid in id_to_idx:
        row_idx = id_to_idx[pid]
        col_idxs = [term_to_idx[t] for t in terms]
        Y_all[row_idx, col_idxs] = 1.0

print(f"X shape: {X_all.shape}, Y shape: {Y_all.shape}")

# 3. Chia Train/Val
X_train, X_val, Y_train, Y_val = train_test_split(X_all, Y_all, test_size=0.1, random_state=42)

# Chuyển sang Tensor
train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(Y_train).float())
val_dataset = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(Y_val).float())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Dọn RAM
del X_all, Y_all, train_terms, grouped
gc.collect()

Đang load dữ liệu lên RAM...
Đang xử lý nhãn...


76297it [00:00, 278218.30it/s]


X shape: (82404, 1280), Y shape: (82404, 1500)


0

In [9]:
#XÂY DỰNG MODEL (MLP)
class ProteinClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ProteinClassifier, self).__init__()
        
        self.network = nn.Sequential(
            # Layer 1
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512), # Giúp hội tụ nhanh
            nn.ReLU(),
            nn.Dropout(0.3),     # Chống học vẹt (Overfitting)
            
            # Layer 2
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # Output Layer
            nn.Linear(256, num_classes)
            # Không dùng Sigmoid ở đây vì BCEWithLogitsLoss đã tích hợp sẵn
        )
        
    def forward(self, x):
        return self.network(x)

model = ProteinClassifier(input_dim=1280, num_classes=TOP_LABELS).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss() # Loss chuẩn cho Multi-label

In [10]:
print("Bắt đầu Train Neural Network...")

best_val_loss = float('inf')

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    
    for batch_x, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    avg_loss = total_loss / len(train_loader)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
            
    avg_val_loss = val_loss / len(val_loader)
    
    print(f"Epoch {epoch+1}: Train Loss = {avg_loss:.4f} | Val Loss = {avg_val_loss:.4f}")
    
    # Save Best Model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print("  --> Saved Best Model")

print("Train xong!")

Bắt đầu Train Neural Network...


Epoch 1: Train Loss = 0.0319 | Val Loss = 0.0140
  --> Saved Best Model


Epoch 2: Train Loss = 0.0139 | Val Loss = 0.0130
  --> Saved Best Model


Epoch 3: Train Loss = 0.0131 | Val Loss = 0.0125
  --> Saved Best Model


Epoch 4: Train Loss = 0.0126 | Val Loss = 0.0121
  --> Saved Best Model


...

Epoch 11: Train Loss = 0.0114 | Val Loss = 0.0113
  --> Saved Best Model


Epoch 12: Train Loss = 0.0113 | Val Loss = 0.0113


Epoch 13: Train Loss = 0.0113 | Val Loss = 0.0112
  --> Saved Best Model


Epoch 14: Train Loss = 0.0112 | Val Loss = 0.0112
  --> Saved Best Model


Epoch 15: Train Loss = 0.0111 | Val Loss = 0.0111
  --> Saved Best Model
Train xong!


In [11]:
print("Đang dự đoán tập Test...")
import glob

# Load lại best model
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.eval()

chunk_files = sorted(glob.glob("/workspace/data/Embeddings/embeddings_chunks/test_part_*.npy"), 
                     key=lambda x: int(x.split('_')[-1].replace('.npy','')))

output_lines = []
# Danh sách tên các nhãn
terms_columns = top_term_list

for f_path in tqdm(chunk_files):
    X_chunk = np.load(f_path)
    # Chuyển sang Tensor
    X_tensor = torch.from_numpy(X_chunk).float().to(device)
    
    id_path = f_path.replace("test_part_", "test_ids_")
    ids_chunk = np.load(id_path, allow_pickle=True)
    
    with torch.no_grad():
        logits = model(X_tensor)
        probs = torch.sigmoid(logits).cpu().numpy() # Chuyển về xác suất 0-1
        
    # Ghi kết quả
    for i, pid in enumerate(ids_chunk):
        prob_row = probs[i]
        # Lấy Top 50 cao nhất
        top_indices = np.argsort(prob_row)[-50:]
        
        for idx in top_indices:
            score = prob_row[idx]
            term = terms_columns[idx]
            if score > 0.01:
                output_lines.append(f"{pid}\t{term}\t{score:.3f}")
                
    del X_chunk, X_tensor
    gc.collect()

Đang dự đoán tập Test...


100%|██████████| 45/45 [00:21<00:00,  2.05it/s]


In [14]:
OUTPUT_FILE = "/workspace/notebooks/submission_mlp_esm2.tsv"
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("HOÀN TẤT")

💾 Đang lưu /workspace/notebooks/submission_mlp_esm2.tsv...
HOÀN TẤT


#### + Propagation

In [15]:
!pip install -q obonet networkx

import networkx
import obonet
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

INPUT_FILE = "/workspace/notebooks/submission_mlp_esm2.tsv"
OUTPUT_FILE = "/workspace/notebooks/submission_mlp_propagated.tsv"

OBO_PATH = "/workspace/data/Train/go-basic.obo"

# --- BƯỚC 1: XÂY DỰNG BẢN ĐỒ TỔ TIÊN ---
print("Đang đọc cấu trúc Gene Ontology...")
graph = obonet.read_obo(OBO_PATH)

print("Đang map quan hệ (Con -> Cha Ông)...")
ancestors_map = {}
# Duyệt qua tất cả các node để tìm tổ tiên trước (Pre-compute)
for node in tqdm(graph.nodes()):
    try:
        # networkx.descendants trả về tập hợp các node cha/ông
        ancestors = networkx.descendants(graph, node)
        ancestors_map[node] = ancestors
    except:
        pass

# --- BƯỚC 2: ĐỌC FILE DỰ ĐOÁN ---
print(f"Đang đọc file {INPUT_FILE}...")
preds = {} # Cấu trúc: {ProteinID: {Term: Score}}

try:
    with open(INPUT_FILE) as f:
        for line in tqdm(f):
            parts = line.strip().split('\t')
            if len(parts) < 3: continue
            pid, term, score = parts[0], parts[1], float(parts[2])
            
            if pid not in preds: preds[pid] = {}
            preds[pid][term] = score
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {INPUT_FILE}. Bạn đã chạy xong bước Train MLP chưa?")
    raise

# --- BƯỚC 3: LAN TRUYỀN (PROPAGATION) ---
print("Đang lan truyền điểm số (Logic: Điểm Cha = Max(Cha cũ, Con))...")
output_lines = []

for pid, term_scores in tqdm(preds.items()):
    # Tạo bản sao để cập nhật điểm
    new_scores = term_scores.copy()
    
    # Duyệt qua từng term con đang có điểm
    for term, score in term_scores.items():
        # Nếu term này có cha ông trong bản đồ
        if term in ancestors_map:
            parents = ancestors_map[term]
            for parent in parents:
                # Cập nhật điểm cho cha
                current_p_score = new_scores.get(parent, 0.0)
                # Chỉ cập nhật nếu điểm mới cao hơn điểm cũ
                if score > current_p_score:
                    new_scores[parent] = score
    
    # --- BƯỚC 4: LỌC TOP K ĐỂ FILE KHÔNG BỊ NẶNG ---
    # Sau khi lan truyền, một protein có thể có hàng trăm nhãn cha.
    # Ta cần lọc lấy Top 75 điểm cao nhất.
    
    # Sắp xếp giảm dần theo điểm
    sorted_terms = sorted(new_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Lấy Top 75
    for term, score in sorted_terms[:75]:
        # Chỉ lấy điểm > 0.01 để bỏ rác
        if score > 0.01:
            output_lines.append(f"{pid}\t{term}\t{score:.3f}")

# --- BƯỚC 5: LƯU FILE ---
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("-" * 30)
print(f"HOÀN THÀNH!")

Đang đọc cấu trúc Gene Ontology...
Đang map quan hệ (Con -> Cha Ông)...


100%|██████████| 40122/40122 [00:00<00:00, 160248.07it/s]


Đang đọc file /workspace/notebooks/submission_mlp_esm2.tsv...


7982025it [00:03, 2331406.83it/s]


Đang lan truyền điểm số (Logic: Điểm Cha = Max(Cha cũ, Con))...


100%|██████████| 224309/224309 [00:12<00:00, 18075.72it/s]


Đang lưu /workspace/notebooks/submission_mlp_propagated.tsv...
------------------------------
HOÀN THÀNH!


#### Score: 0.204 -> Học vẹt ?

### Ensemble: KNN + BLAST

In [17]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# --- CẤU HÌNH ---
# 1. File KNN tốt nhất của bạn (File đạt 0.234)
# Lưu ý: Chọn đúng file output của bước KNN có Taxonomy + Propagation
KNN_FILE = "/workspace/notebooks/submission_knn_tax_propagated.tsv" 

# 2. File kết quả BLAST (Level 2)
# Nếu bạn chưa chạy BLAST hoặc đã xóa, đoạn code dưới sẽ tự chạy lại (mất 3 phút)
BLAST_RESULT = "/workspace/data/diamond_results.tsv"
TRAIN_FASTA = "/workspace/data/Train/train_sequences.fasta"
TEST_FASTA = "/workspace/data/Test/testsuperset.fasta"
DB_PATH = "/workspace/data/Train/train_data.dmnd"
DIAMOND_BIN = "/usr/bin/diamond" # Hoặc "diamond" nếu đã add path

OUTPUT_FILE = "/workspace/notebooks/submission_hybrid_best_v1.tsv"

# --- BƯỚC 1: KIỂM TRA VÀ CHẠY BLAST NẾU CẦN ---
if not os.path.exists(BLAST_RESULT):
    print("Không thấy file BLAST cũ, đang chạy lại...")
    if not os.path.exists(DB_PATH):
        os.system(f"{DIAMOND_BIN} makedb --in {TRAIN_FASTA} -d {DB_PATH} --quiet")
    
    # Chạy so khớp
    cmd = f"{DIAMOND_BIN} blastp -d {DB_PATH} -q {TEST_FASTA} -o {BLAST_RESULT} --sensitive --top 1 -f 6 qseqid sseqid pident"
    os.system(cmd)
    print("BLAST xong!")
else:
    print("Đã có sẵn file BLAST.")

# --- BƯỚC 2: ĐỌC DỮ LIỆU ---
print("Đang load dữ liệu...")

# 1. Load BLAST Predictions
# Logic: Chỉ tin tưởng BLAST nếu độ giống nhau > 30%
# Map TrainID sang Terms
train_terms = pd.read_csv("/workspace/data/Train/train_terms.tsv", sep="\t", usecols=["EntryID", "term"])
train_terms_grouped = train_terms.groupby("EntryID")["term"].apply(list).to_dict()

blast_preds = {}
df_blast = pd.read_csv(BLAST_RESULT, sep="\t", names=["test_id", "train_id", "pident"])

print("Đang xử lý BLAST...")
for _, row in df_blast.iterrows():
    pident = row['pident'] / 100.0
    if pident < 0.3: continue # Bỏ qua nếu giống nhau quá ít
    
    test_id = str(row['test_id']).split("|")[1] if "|" in str(row['test_id']) else str(row['test_id'])
    train_id = str(row['train_id']).split("|")[1] if "|" in str(row['train_id']) else str(row['train_id'])
    
    if train_id in train_terms_grouped:
        for term in train_terms_grouped[train_id]:
            # BLAST score rất uy tín
            blast_preds[(test_id, term)] = pident

# 2. Load KNN Predictions (File 0.234)
print(f"Đang đọc file KNN: {KNN_FILE}...")
knn_preds = {}
try:
    with open(KNN_FILE) as f:
        for line in tqdm(f):
            parts = line.strip().split('\t')
            if len(parts) < 3: continue
            pid, term, score = parts[0], parts[1], float(parts[2])
            knn_preds[(pid, term)] = score
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {KNN_FILE}. Bạn hãy chắc chắn đã chạy xong bước KNN Tax Propagation.")
    raise

# --- BƯỚC 3: TRỘN (MAX STRATEGY) ---
print("Đang trộn BLAST + KNN (Lấy điểm cao nhất)...")
# Logic: Final_Score = Max(BLAST, KNN)
# Nếu BLAST tìm thấy (độ tin cậy cao), nó sẽ ghi đè lên KNN.
# Nếu BLAST bó tay, ta dùng KNN.

all_keys = set(blast_preds.keys()) | set(knn_preds.keys())
output_lines = []

for key in tqdm(all_keys):
    pid, term = key
    
    s_blast = blast_preds.get(key, 0.0)
    s_knn = knn_preds.get(key, 0.0)
    
    final_score = max(s_blast, s_knn)
    
    # Lọc rác lần cuối
    if final_score > 0.01:
        output_lines.append(f"{pid}\t{term}\t{final_score:.3f}")

# --- BƯỚC 4: LƯU FILE ---
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("XONG! Hãy nộp file này.")

Đã có sẵn file BLAST.
Đang load dữ liệu...
Đang xử lý BLAST...
Đang đọc file KNN: /workspace/notebooks/submission_knn_tax_propagated.tsv...


16226847it [00:10, 1523809.83it/s]


Đang trộn BLAST + KNN (Lấy điểm cao nhất)...


100%|██████████| 16710313/16710313 [00:19<00:00, 849071.23it/s]


Đang lưu /workspace/notebooks/submission_hybrid_best_v1.tsv...
XONG! Hãy nộp file này.


#### Score: 0.225

### MLP with early stopping

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gc
import os

# --- CẤU HÌNH ---
TRAIN_EMB_PATH = "/workspace/data/Embeddings/train_650M.npy"
TRAIN_IDS_PATH = "/workspace/data/Embeddings/train_650M_ids.npy"
TRAIN_TERMS_PATH = "/workspace/data/Train/train_terms.tsv"
MODEL_SAVE_PATH = "/workspace/models/best_mlp_model.pth"
OUTPUT_FILE = "/workspace/notebooks/submission_mlp_tuned.tsv"

BATCH_SIZE = 256 # Tăng batch size để gradient ổn định hơn
EPOCHS = 50      # Cho max 50, nhưng Early Stop sẽ dừng sớm
LEARNING_RATE = 5e-4 # Giảm LR xuống để học chậm mà chắc (tránh vọt xà)
TOP_LABELS = 1500 
PATIENCE = 5     # Cho phép không tiến bộ 5 lần, quá 5 lần là cắt

device = "cuda" if torch.cuda.is_available() else "cpu"

# --- BƯỚC 1: LOAD DỮ LIỆU (Giống hệt cũ) ---
print("Đang load dữ liệu...")
X_all = np.load(TRAIN_EMB_PATH)
ids_all = np.load(TRAIN_IDS_PATH, allow_pickle=True)

train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep="\t", usecols=["EntryID", "term"])
ids_set = set(ids_all)
train_terms = train_terms[train_terms["EntryID"].isin(ids_set)]
top_term_list = train_terms["term"].value_counts().head(TOP_LABELS).index.tolist()
term_to_idx = {t: i for i, t in enumerate(top_term_list)}

Y_all = np.zeros((len(ids_all), TOP_LABELS), dtype=np.float32)
id_to_idx = {pid: i for i, pid in enumerate(ids_all)}
grouped = train_terms[train_terms["term"].isin(top_term_list)].groupby("EntryID")["term"].apply(list)
for pid, terms in grouped.items():
    if pid in id_to_idx:
        Y_all[id_to_idx[pid], [term_to_idx[t] for t in terms]] = 1.0

# Chia tập (Quan trọng: Stratify để chia đều nhãn khó/dễ nếu có thể, nhưng multilabel khó stratify chuẩn nên random)
X_train, X_val, Y_train, Y_val = train_test_split(X_all, Y_all, test_size=0.15, random_state=42) # Tăng Val lên 15%

train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(Y_train).float())
val_dataset = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(Y_val).float())
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

del X_all, Y_all, train_terms
gc.collect()

# --- BƯỚC 2: MODEL CẢI TIẾN ---
class ProteinClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ProteinClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1024), # Tăng layer đầu
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),     # Dropout mạnh tay (50%) để phạt học vẹt
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        return self.network(x)

model = ProteinClassifier(1280, TOP_LABELS).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4) # AdamW chống overfitting tốt hơn
criterion = nn.BCEWithLogitsLoss()

# --- BƯỚC 3: TRAIN VỚI EARLY STOPPING ---
print("Bắt đầu Train (Có Early Stopping)...")
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for bx, by in train_loader:
        bx, by = bx.to(device), by.to(device)
        optimizer.zero_grad()
        loss = criterion(model(bx), by)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for bx, by in val_loader:
            bx, by = bx.to(device), by.to(device)
            val_loss += criterion(model(bx), by).item()
            
    avg_train = train_loss / len(train_loader)
    avg_val = val_loss / len(val_loader)
    
    print(f"Epoch {epoch+1}: Train={avg_train:.4f} | Val={avg_val:.4f}", end="")
    
    # LOGIC EARLY STOPPING
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        patience_counter = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(" --> Saved Best Model")
    else:
        patience_counter += 1
        print(f" | Patience {patience_counter}/{PATIENCE}")
        
    if patience_counter >= PATIENCE:
        print(f"DỪNG SỚM! Model bắt đầu học vẹt tại Epoch {epoch+1}")
        break

# --- BƯỚC 4: DỰ ĐOÁN & GHI FILE ---
# (Phần này giữ nguyên logic cũ, chỉ load lại best model)
print("Đang dự đoán tập Test với Best Model...")
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.eval()

import glob
from tqdm import tqdm
chunk_files = sorted(glob.glob("/workspace/data/Embeddings/embeddings_chunks/test_part_*.npy"), 
                     key=lambda x: int(x.split('_')[-1].replace('.npy','')))

output_lines = []
for f_path in tqdm(chunk_files):
    X_chunk = np.load(f_path)
    X_tensor = torch.from_numpy(X_chunk).float().to(device)
    ids_chunk = np.load(f_path.replace("test_part_", "test_ids_"), allow_pickle=True)
    
    with torch.no_grad():
        probs = torch.sigmoid(model(X_tensor)).cpu().numpy()
        
    for i, pid in enumerate(ids_chunk):
        # Lấy Top 50, ngưỡng thấp hơn 1 chút
        top_idxs = np.argsort(probs[i])[-50:]
        for idx in top_idxs:
            s = probs[i][idx]
            if s > 0.005: # Hạ threshold xuống chút
                output_lines.append(f"{pid}\t{top_term_list[idx]}\t{s:.3f}")
                
    del X_chunk, X_tensor
    gc.collect()

with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))
print("Done! Hãy chạy tiếp Propagation cho file 'submission_mlp_tuned.tsv'")

Đang load dữ liệu...
Bắt đầu Train (Có Early Stopping)...
Epoch 1: Train=0.0540 | Val=0.0158 --> Saved Best Model
Epoch 2: Train=0.0149 | Val=0.0142 --> Saved Best Model
Epoch 3: Train=0.0141 | Val=0.0136 --> Saved Best Model
Epoch 4: Train=0.0136 | Val=0.0132 --> Saved Best Model
Epoch 5: Train=0.0132 | Val=0.0129 --> Saved Best Model
Epoch 6: Train=0.0129 | Val=0.0126 --> Saved Best Model
Epoch 7: Train=0.0126 | Val=0.0124 --> Saved Best Model
Epoch 8: Train=0.0124 | Val=0.0123 --> Saved Best Model
Epoch 9: Train=0.0122 | Val=0.0121 --> Saved Best Model
Epoch 10: Train=0.0121 | Val=0.0119 --> Saved Best Model
Epoch 11: Train=0.0119 | Val=0.0119 | Patience 1/5
Epoch 12: Train=0.0118 | Val=0.0118 --> Saved Best Model
Epoch 13: Train=0.0117 | Val=0.0118 | Patience 1/5
Epoch 14: Train=0.0116 | Val=0.0116 --> Saved Best Model
Epoch 15: Train=0.0115 | Val=0.0115 --> Saved Best Model
Epoch 16: Train=0.0114 | Val=0.0115 --> Saved Best Model
Epoch 17: Train=0.0114 | Val=0.0115 | Patience 1/5


100%|██████████| 45/45 [00:23<00:00,  1.95it/s]


Done! Hãy chạy tiếp Propagation cho file 'submission_mlp_tuned.tsv'


In [5]:
%pip install -q obonet networkx

import networkx
import obonet
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# --- CẤU HÌNH ---
# 1. File đầu vào: Kết quả của MLP Tuned vừa chạy
INPUT_FILE = "/workspace/notebooks/submission_mlp_tuned.tsv"

# 2. File đầu ra: File nộp cuối cùng
OUTPUT_FILE = "/workspace/notebooks/submission_mlp_tuned_propagated.tsv"

# 3. Tìm file OBO (Cây phả hệ)
if os.path.exists("/workspace/data/Train/go-basic.obo"):
    OBO_PATH = "/workspace/data/Train/go-basic.obo"
else:
    OBO_PATH = "/workspace/data/go-basic.obo"

print(f"Đang dùng file OBO tại: {OBO_PATH}")

# --- BƯỚC 1: XÂY DỰNG BẢN ĐỒ TỔ TIÊN ---
print("Đang đọc cấu trúc Gene Ontology...")
graph = obonet.read_obo(OBO_PATH)

print("Đang map quan hệ (Con -> Cha Ông)...")
ancestors_map = {}
# Duyệt qua tất cả các node để tìm tổ tiên (Pre-compute cho nhanh)
for node in tqdm(graph.nodes()):
    try:
        # networkx.descendants trả về tập hợp các node cha/ông
        ancestors = networkx.descendants(graph, node)
        ancestors_map[node] = ancestors
    except:
        pass

# --- BƯỚC 2: ĐỌC FILE DỰ ĐOÁN ---
print(f"Đang đọc file {INPUT_FILE}...")
preds = {} # Cấu trúc: {ProteinID: {Term: Score}}

try:
    with open(INPUT_FILE) as f:
        for line in tqdm(f):
            parts = line.strip().split('\t')
            if len(parts) < 3: continue
            pid, term, score = parts[0], parts[1], float(parts[2])
            
            if pid not in preds: preds[pid] = {}
            preds[pid][term] = score
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {INPUT_FILE}. Bạn đã chạy xong bước Train MLP Tuned chưa?")
    raise

# --- BƯỚC 3: LAN TRUYỀN (PROPAGATION) ---
print("Đang lan truyền điểm số (Logic: Điểm Cha = Max(Cha cũ, Con))...")
output_lines = []

for pid, term_scores in tqdm(preds.items()):
    # Tạo bản sao để cập nhật điểm
    new_scores = term_scores.copy()
    
    # Duyệt qua từng term con đang có điểm
    for term, score in term_scores.items():
        # Nếu term này có cha ông trong bản đồ
        if term in ancestors_map:
            parents = ancestors_map[term]
            for parent in parents:
                # Cập nhật điểm cho cha
                current_p_score = new_scores.get(parent, 0.0)
                # Chỉ cập nhật nếu điểm mới cao hơn điểm cũ
                if score > current_p_score:
                    new_scores[parent] = score
    
    # --- BƯỚC 4: LỌC & GHI FILE ---
    # Sắp xếp giảm dần theo điểm
    sorted_terms = sorted(new_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Lấy Top 75 điểm cao nhất (để file không quá nặng)
    for term, score in sorted_terms[:75]:
        # Chỉ lấy điểm > 0.005
        if score > 0.005:
            output_lines.append(f"{pid}\t{term}\t{score:.3f}")

# --- BƯỚC 5: LƯU FILE ---
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("-" * 30)
print(f"HOÀN THÀNH! File '{OUTPUT_FILE}' đã sẵn sàng.")
print("Hãy nộp file này lên Kaggle và xem sự khác biệt!")

Note: you may need to restart the kernel to use updated packages.
Đang dùng file OBO tại: /workspace/data/Train/go-basic.obo
Đang đọc cấu trúc Gene Ontology...
Đang map quan hệ (Con -> Cha Ông)...


100%|██████████| 40122/40122 [00:00<00:00, 154454.33it/s]


Đang đọc file /workspace/notebooks/submission_mlp_tuned.tsv...


9935666it [00:04, 2323627.72it/s]


Đang lan truyền điểm số (Logic: Điểm Cha = Max(Cha cũ, Con))...


100%|██████████| 224309/224309 [00:14<00:00, 14988.68it/s]


Đang lưu /workspace/notebooks/submission_mlp_tuned_propagated.tsv...
------------------------------
HOÀN THÀNH! File '/workspace/notebooks/submission_mlp_tuned_propagated.tsv' đã sẵn sàng.
Hãy nộp file này lên Kaggle và xem sự khác biệt!


#### Score: 0.217

### Taxonomy-aware KNN + BLAST

In [6]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# --- CẤU HÌNH ---
# 1. File KNN xịn nhất (File đạt 0.234)
# (Đảm bảo bạn dùng đúng file đã chạy Propagation)
KNN_FILE = "/workspace/notebooks/submission_knn_tax_propagated.tsv"

# 2. Cấu hình để chạy lại BLAST (nếu cần)
BLAST_RESULT = "/workspace/data/diamond_results.tsv"
TRAIN_FASTA = "/workspace/data/Train/train_sequences.fasta"
TEST_FASTA = "/workspace/data/Test/testsuperset.fasta"
DB_PATH = "/workspace/data/Train/train_data.dmnd"
DIAMOND_BIN = "/usr/bin/diamond" # Hoặc đường dẫn tới file diamond

OUTPUT_FILE = "/workspace/notebooks/submission_FINAL_ENSEMBLE.tsv"

# --- BƯỚC 1: ĐẢM BẢO CÓ KẾT QUẢ BLAST ---
if not os.path.exists(BLAST_RESULT):
    print("Đang chạy lại BLAST lần cuối (cho chắc ăn)...")
    if not os.path.exists(DB_PATH):
        os.system(f"{DIAMOND_BIN} makedb --in {TRAIN_FASTA} -d {DB_PATH} --quiet")
    
    # Chạy Diamond (Sensitive mode)
    cmd = f"{DIAMOND_BIN} blastp -d {DB_PATH} -q {TEST_FASTA} -o {BLAST_RESULT} --sensitive --top 1 -f 6 qseqid sseqid pident"
    os.system(cmd)
else:
    print("Đã có sẵn kết quả BLAST.")

# --- BƯỚC 2: LOAD DỮ LIỆU BLAST ---
print("Đang xử lý BLAST...")
# Load mapping ID -> Terms
train_terms = pd.read_csv("/workspace/data/Train/train_terms.tsv", sep="\t", usecols=["EntryID", "term"])
train_terms_grouped = train_terms.groupby("EntryID")["term"].apply(list).to_dict()

blast_preds = {}
df_blast = pd.read_csv(BLAST_RESULT, sep="\t", names=["test_id", "train_id", "pident"])

for _, row in tqdm(df_blast.iterrows(), total=len(df_blast)):
    pident = row['pident'] / 100.0
    # Chỉ tin tưởng BLAST nếu độ giống > 35%
    if pident < 0.35: continue
    
    # Clean ID
    tid = str(row['test_id']).split("|")[1] if "|" in str(row['test_id']) else str(row['test_id'])
    trid = str(row['train_id']).split("|")[1] if "|" in str(row['train_id']) else str(row['train_id'])
    
    if trid in train_terms_grouped:
        for term in train_terms_grouped[trid]:
            # Lưu điểm BLAST
            blast_preds[(tid, term)] = pident

# --- BƯỚC 3: LOAD DỮ LIỆU KNN (0.234) ---
print(f"Đang đọc file KNN tốt nhất ({KNN_FILE})...")
knn_preds = {}
try:
    with open(KNN_FILE) as f:
        for line in tqdm(f):
            parts = line.strip().split('\t')
            if len(parts) < 3: continue
            knn_preds[(parts[0], parts[1])] = float(parts[2])
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {KNN_FILE}. Bạn check lại tên file KNN 0.234 nhé.")
    raise

# --- BƯỚC 4: HỢP THỂ (MAX STRATEGY) ---
print("Đang trộn (Max Strategy)...")
all_keys = set(blast_preds.keys()) | set(knn_preds.keys())
output_lines = []

for key in tqdm(all_keys):
    pid, term = key
    
    s_blast = blast_preds.get(key, 0.0)
    s_knn = knn_preds.get(key, 0.0)
    
    # Lấy điểm cao nhất giữa 2 thuật toán
    final_score = max(s_blast, s_knn)
    
    if final_score > 0.01:
        output_lines.append(f"{pid}\t{term}\t{final_score:.3f}")

# --- BƯỚC 5: LƯU FILE ---
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("DONE!")

Đã có sẵn kết quả BLAST.
Đang xử lý BLAST...


100%|██████████| 262530/262530 [00:03<00:00, 74232.65it/s]


Đang đọc file KNN tốt nhất (/workspace/notebooks/submission_knn_tax_propagated.tsv)...


16226847it [00:09, 1634421.68it/s]


Đang trộn (Max Strategy)...


100%|██████████| 16693341/16693341 [00:18<00:00, 904218.13it/s]


Đang lưu /workspace/notebooks/submission_FINAL_ENSEMBLE.tsv...
DONE!


#### Score: 0.224

### Weighted F1: Nhãn hiếm -> điểm cao. Nhưng KNN đang xử lí các nhãn là như nhau

### Sol: IC weighting (Information Content weighting): Nhãn nào hiếm -> bơm điểm

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math

# --- CẤU HÌNH ---
# Dùng lại file tốt nhất của bạn (File đạt 0.234)
# (Đảm bảo file này đang nằm trong thư mục hiện tại)
INPUT_FILE = "submission_knn_tax_propagated.tsv"
OUTPUT_FILE = "submission_knn_tax_IC_weighted.tsv"
TRAIN_TERMS = "/workspace/data/Train/train_terms.tsv"

# --- BƯỚC 1: TÍNH ĐỘ HIẾM (INFORMATION CONTENT) ---
print("Đang tính toán độ hiếm của từng nhãn (IC)...")
df_terms = pd.read_csv(TRAIN_TERMS, sep="\t", usecols=['term'])
term_counts = df_terms['term'].value_counts().to_dict()
total_proteins = len(df_terms)

# Hàm tính IC: Càng hiếm càng cao
# IC = -log2(Count / Total)
term_weights = {}
for term, count in term_counts.items():
    prob = count / total_proteins
    ic = -math.log2(prob)
    # Chuẩn hóa nhẹ để không bơm quá lố (Scale về khoảng 1.0 - 2.0)
    # Nhãn cực hiếm (IC cao) sẽ được nhân hệ số lớn
    term_weights[term] = ic

print(f"✅ Đã tính trọng số cho {len(term_weights)} nhãn.")

# --- BƯỚC 2: BƠM ĐIỂM CHO FILE CŨ ---
print(f"Đang đọc và boost điểm cho {INPUT_FILE}...")
output_lines = []

with open(INPUT_FILE) as f:
    for line in tqdm(f):
        parts = line.strip().split('\t')
        if len(parts) < 3: continue
        pid, term, score = parts[0], parts[1], float(parts[2])
        
        # Lấy trọng số IC (Nếu không có trong train thì mặc định là hiếm -> weight cao)
        # Weight trung bình thường tầm 5-10. Ta scale xuống chút cho an toàn.
        weight = term_weights.get(term, 10.0) 
        
        # LOGIC BOOST:
        # Score mới = Score cũ * (1 + Factor * IC)
        # Factor 0.05 nghĩa là nhãn hiếm được thưởng thêm tầm 20-50% điểm
        boost_factor = 0.05 
        new_score = score * (1 + boost_factor * weight)
        
        # Clip về 1.0
        new_score = min(new_score, 1.0)
        
        # Chỉ giữ lại nếu điểm số đủ cao (Lọc rác kỹ hơn)
        if new_score > 0.015:
            output_lines.append(f"{pid}\t{term}\t{new_score:.3f}")

# --- BƯỚC 3: LƯU FILE ---
print(f"Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("XONG!")

Đang tính toán độ hiếm của từng nhãn (IC)...
✅ Đã tính trọng số cho 26125 nhãn.
Đang đọc và boost điểm cho submission_knn_tax_propagated.tsv...


16226847it [00:11, 1391612.00it/s]


Đang lưu submission_knn_tax_IC_weighted.tsv...
XONG!


#### Score: 0.24

#### Vét

In [2]:
import pandas as pd
from tqdm import tqdm
import math

# --- CẤU HÌNH ---
# Dùng file KNN gốc (File 0.234 - trước khi IC weight)
# Lưu ý: Phải là file "submission_knn_tax_propagated.tsv"
INPUT_FILE = "submission_knn_tax_propagated.tsv"
TRAIN_TERMS = "/workspace/data/Train/train_terms.tsv"

# --- BƯỚC 1: TÍNH LẠI IC (Giữ nguyên) ---
print("Đang tính IC...")
df_terms = pd.read_csv(TRAIN_TERMS, sep="\t", usecols=['term'])
term_counts = df_terms['term'].value_counts().to_dict()
total_proteins = len(df_terms)
term_weights = {}
for term, count in term_counts.items():
    term_weights[term] = -math.log2(count / total_proteins)

# --- BƯỚC 2: TẠO 3 BIẾN THỂ ---
# Cấu hình các chiến thuật
strategies = [
    {"name": "v1_conservative", "boost": 0.02, "thr": 0.015}, # Bơm nhẹ
    {"name": "v2_aggressive",   "boost": 0.10, "thr": 0.015}, # Bơm mạnh tay
    {"name": "v3_loose_thr",    "boost": 0.05, "thr": 0.010}, # Bơm vừa, nhưng lấy nhiều nhãn hơn
]

print("Đang tạo 3 file biến thể...")

# Đọc file gốc vào bộ nhớ cho nhanh
with open(INPUT_FILE) as f:
    lines = f.readlines()

for strat in strategies:
    out_name = f"submission_IC_{strat['name']}.tsv"
    boost = strat['boost']
    thr = strat['thr']
    
    print(f"-> Tạo {out_name} (Boost={boost}, Thr={thr})...")
    out_lines = []
    
    for line in tqdm(lines, leave=False):
        parts = line.strip().split('\t')
        if len(parts) < 3: continue
        pid, term, score = parts[0], parts[1], float(parts[2])
        
        weight = term_weights.get(term, 10.0)
        
        # Công thức boost
        new_score = score * (1 + boost * weight)
        new_score = min(new_score, 1.0)
        
        if new_score > thr:
            out_lines.append(f"{pid}\t{term}\t{new_score:.3f}")
            
    with open(out_name, "w") as f:
        f.write("\n".join(out_lines))

print("Đã tạo xong 3 file! Hãy nộp lần lượt để tìm ra cấu hình tốt nhất.")

Đang tính IC...
Đang tạo 3 file biến thể...
-> Tạo submission_IC_v1_conservative.tsv (Boost=0.02, Thr=0.015)...


-> Tạo submission_IC_v2_aggressive.tsv (Boost=0.1, Thr=0.015)...


-> Tạo submission_IC_v3_loose_thr.tsv (Boost=0.05, Thr=0.01)...


Đã tạo xong 3 file! Hãy nộp lần lượt để tìm ra cấu hình tốt nhất.
